In [1]:
from copy import deepcopy
from pprint import pprint

In [2]:
def merge_learning_objects(existing_data, new_data):
    data = deepcopy(existing_data)
    for kl in data:
        for question_type in data[kl]:
            data[kl][question_type]['num_questions'] += new_data[kl][question_type]['num_questions']
            data[kl][question_type]['total_score'] += new_data[kl][question_type]['total_score']
    return data

In [3]:
def prepare_matrix_data(json_object):
    lo_data = {
        'knowing': {
            'multiple_choice': {
                'num_questions': 0,
                'total_score': 0
            },
            'free_form': {
                'num_questions': 0,
                'total_score': 0
            }
        },
        'understanding': {
            'multiple_choice': {
                'num_questions': 0,
                'total_score': 0
            },
            'free_form': {
                'num_questions': 0,
                'total_score': 0
            }
        },
        'application': {
            'multiple_choice': {
                'num_questions': 0,
                'total_score': 0
            },
            'free_form': {
                'num_questions': 0,
                'total_score': 0
            }
        },
        'adv_application': {
            'multiple_choice': {
                'num_questions': 0,
                'total_score': 0
            },
            'free_form': {
                'num_questions': 0,
                'total_score': 0
            }
        }
    }

    question_types = {
        'Trắc nghiệm': 'multiple_choice',
        'Tự luận': 'free_form'
    }

    knowledge_levels = {
        'Nhận biết': 'knowing',
        'Thông hiểu': 'understanding',
        'Vận dụng': 'application',
        'Vận dụng cao': 'adv_application'
    }

    matrix_data = {}

    for item in json_object:
        chapter = item["Chương hoặc Chủ đề"]
        learning_objective = item["Nội dung hoặc Đơn vị Kiến thức"]
        mastery_level = item["Mức độ Nhận thức"]
        question_type = item["Loại câu hỏi"]
        question_num = item["Số câu hỏi"]
        total_score = item["Tổng số điểm"]

        lo_question = deepcopy(lo_data)

        lo_question[knowledge_levels[mastery_level]][question_types[question_type]]['num_questions'] = int(question_num)
        lo_question[knowledge_levels[mastery_level]][question_types[question_type]]['total_score'] = int(total_score)

        if chapter not in matrix_data.keys():
            matrix_data[chapter] = {learning_objective: lo_question}
        else:
            if learning_objective not in matrix_data[chapter].keys():
                matrix_data[chapter][learning_objective] = lo_question
            else:
                matrix_data[chapter][learning_objective] = merge_learning_objects(matrix_data[chapter][learning_objective], lo_question)

    return matrix_data

In [4]:
def percentage_calculation(matrix_data):
    total = {
            'knowing': {
                'multiple_choice': {
                    'total_questions': 0,
                    'total_score': 0
                },
                'free_form': {
                    'total_questions': 0,
                    'total_score': 0
                }
            },
            'understanding': {
                'multiple_choice': {
                    'total_questions': 0,
                    'total_score': 0
                },
                'free_form': {
                    'total_questions': 0,
                    'total_score': 0
                }
            },
            'application': {
                'multiple_choice': {
                    'total_questions': 0,
                    'total_score': 0
                },
                'free_form': {
                    'total_questions': 0,
                    'total_score': 0
                }
            },
            'adv_application': {
                'multiple_choice': {
                    'total_questions': 0,
                    'total_score': 0
                },
                'free_form': {
                    'total_questions': 0,
                    'total_score': 0
                }
            }
        }

    for chapter in matrix_data.keys():
        for learning_objective in matrix_data[chapter].keys():
            for kl in matrix_data[chapter][learning_objective].keys():
                for question in matrix_data[chapter][learning_objective][kl].keys():
                    total[kl][question]['total_questions'] += matrix_data[chapter][learning_objective][kl][question]['num_questions']
                    total[kl][question]['total_score'] += matrix_data[chapter][learning_objective][kl][question]['total_score']

    return total

In [5]:
def generating_exam_matrix(matrix_json_obj):
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="utf-8" />
        <meta name="viewport" content="width=device-width, initial-scale=1" />
        <title></title>
        <style>
        html {
            font-size: 10px;
        }
        body {
            font-family: Georgia, Arial, Helvetica, sans-serif;
            font-size: 1.6rem;
        }
        h1 {
            font-size: 2rem;
            text-align: center;
            padding: 1rem 0;
        }
        table {
            width: 100%;
            border-style: solid;
            border-color:#000;
            border-width: 1px 0 0 1px;
            border-spacing: 0px;
        }
        td, th {
            border: 1px solid #000;
            border-width: 0 1px 1px 0;
            padding: .25rem .5rem;
        }
        thead th {
            background-color: #f0f0f0;
        }
        tbody td:first-child {
            text-align: left;
        }
        tfoot td {
            text-align: center;
            font-weight: bold;
            background-color: #f0f0f0;
        }

        .columnDataNumber {
            text-align: right;
        }
        .borderBottomDashed {
            border-bottom: 1px dashed #000;
        }
        </style>
        <meta name="description" content="" />
    </head>

    <body>
        <h1>
        MA TRẬN KIỂM TRA GIỮA HỌC KỲ I - NĂM HỌC 2023 - 2024
        <br>
        MÔN TOÁN 9 - THỜI GIAN LÀM BÀI 90 PHÚT
        </h1>

        <!-- NOTE: The table has 14 columns -->
        <table>
        <thead>
            <tr>
            <th rowspan="3">TT</th>
            <th rowspan="3">Nội dung Kiến thức</th>
            <th rowspan="3">Đơn vị Kiến thức</th>
            <th colspan="8">Mức độ Nhận thức</th>
            <!-- <th></th>
            <th></th>
            <th></th>
            <th></th>
            <th></th>
            <th></th>
            <th></th> -->
            <th colspan="2">Tổng</th>
            <!-- <th></th> -->
            <th rowspan="3" class="columnDataNumber">Tổng điểm</th>
            </tr>
            <tr>
            <!-- <th rowspan="3">TT</th>
            <th rowspan="3">Nội dung Kiến thức</th>
            <th rowspan="3">Đơn vị Kiến thức</th> -->
            <th colspan="2">Nhận biết</th>
            <!--<th></th> -->
            <th colspan="2">Thông hiểu</th>
            <!--<th></th> -->
            <th colspan="2">Vận dụng thấp</th>
            <!--<th></th> -->
            <th colspan="2">Vận dụng cao</th>
            <!--<th></th> -->
            <th colspan="2">Số câu hỏi</th>
            <!--<th></th> -->
            <!-- <th rowspan="3">Tổng điểm</th> -->
            </tr>
            <tr>
            <!-- <th rowspan="3">TT</th>
            <th rowspan="3">Nội dung Kiến thức</th>
            <th rowspan="3">Đơn vị Kiến thức</th> -->
            <th>TN</th>
            <th>TL</th>
            <th>TN</th>
            <th>TL</th>
            <th>TN</th>
            <th>TL</th>
            <th>TN</th>
            <th>TL</th>
            <th>TN</th>
            <th>TL</th>
            <!-- <th rowspan="3">Tổng điểm</th> -->
            </tr>
        </thead>
                <tbody>
            """
    matrix_data = prepare_matrix_data(data)
    pprint(matrix_data)

    for seq, chapter in enumerate(matrix_data.keys()):
        html_content += f"""
        <tr>
    <td rowspan="{len(matrix_data[chapter]) * 2}">{seq + 1}</td>
    <td rowspan="{len(matrix_data[chapter]) * 2}">{chapter}</td>
        """
        for lo in matrix_data[chapter].keys():
            html_content += f"""
            <td rowspan="2">{lo}</td>
            <td class="columnDataNumber borderBottomDashed">{matrix_data[chapter][lo]['knowing']['multiple_choice']['num_questions']}</td>
            <td class="columnDataNumber borderBottomDashed">{matrix_data[chapter][lo]['knowing']['free_form']['num_questions']}</td>
            <td class="columnDataNumber borderBottomDashed">{matrix_data[chapter][lo]['understanding']['multiple_choice']['num_questions']}</td>
            <td class="columnDataNumber borderBottomDashed">{matrix_data[chapter][lo]['understanding']['free_form']['num_questions']}</td>
            <td class="columnDataNumber borderBottomDashed">{matrix_data[chapter][lo]['application']['multiple_choice']['num_questions']}</td>
            <td class="columnDataNumber borderBottomDashed">{matrix_data[chapter][lo]['application']['free_form']['num_questions']}</td>
            <td class="columnDataNumber borderBottomDashed">{matrix_data[chapter][lo]['adv_application']['multiple_choice']['num_questions']}</td>
            <td class="columnDataNumber borderBottomDashed">{matrix_data[chapter][lo]['adv_application']['free_form']['num_questions']}</td>
            <td class="columnDataNumber borderBottomDashed">{sum([matrix_data[chapter][lo][level]['multiple_choice']['num_questions'] for level in ['knowing', 'understanding', 'application', 'adv_application']])}</td>
            <td class="columnDataNumber borderBottomDashed">{sum([matrix_data[chapter][lo][level]['free_form']['num_questions'] for level in ['knowing', 'understanding', 'application', 'adv_application']])}</td>
            <td class="columnDataNumber borderBottomDashed"></td>
            </tr>
            <tr>
            <!-- <td rowspan="4">1</td>
            <td rowspan="4">Căn bậc hai, căn bậc ba</td>
            <td rowspan="2">Căn bậc hai</td> -->
            <td class="columnDataNumber">{matrix_data[chapter][lo]['knowing']['multiple_choice']['total_score']}đ</td>
            <td class="columnDataNumber">{matrix_data[chapter][lo]['knowing']['free_form']['total_score']}đ</td>
            <td class="columnDataNumber">{matrix_data[chapter][lo]['understanding']['multiple_choice']['total_score']}đ</td>
            <td class="columnDataNumber">{matrix_data[chapter][lo]['understanding']['free_form']['total_score']}đ</td>
            <td class="columnDataNumber">{matrix_data[chapter][lo]['application']['multiple_choice']['total_score']}đ</td>
            <td class="columnDataNumber">{matrix_data[chapter][lo]['application']['free_form']['total_score']}đ</td>
            <td class="columnDataNumber">{matrix_data[chapter][lo]['adv_application']['multiple_choice']['total_score']}đ</td>
            <td class="columnDataNumber">{matrix_data[chapter][lo]['adv_application']['free_form']['total_score']}đ</td>
            <td class="columnDataNumber"></td>
            <td class="columnDataNumber"></td>
            <td class="columnDataNumber">{sum([matrix_data[chapter][lo][level]['multiple_choice']['total_score'] for level in ['knowing', 'understanding', 'application', 'adv_application']]) +  sum([matrix_data[chapter][lo][level]['free_form']['total_score'] for level in ['knowing', 'understanding', 'application', 'adv_application']])} </td>
            </tr>
            """
    total = percentage_calculation(matrix_data)

    total_score = sum([
        total[kl]['multiple_choice']['total_score'] for kl in
        ['knowing', 'understanding', 'application', 'adv_application']
    ]) + sum([
        total[kl]['free_form']['total_score'] for kl in
        ['knowing', 'understanding', 'application', 'adv_application']
    ])
    knowing_score = sum([
        total['knowing'][q]['total_score']
        for q in ['multiple_choice', 'free_form']
    ])
    understanding_score = sum([
        total['understanding'][q]['total_score']
        for q in ['multiple_choice', 'free_form']
    ])
    application_score = sum([
        total['application'][q]['total_score']
        for q in ['multiple_choice', 'free_form']
    ])
    adv_application_score = sum([
        total['adv_application'][q]['total_score']
        for q in ['multiple_choice', 'free_form']
    ])

    html_content += f"""
            </tbody>
            <tfoot>
            <tr>
            <td colspan="3">Tổng</td>
            <!-- <td></td>
            <td></td> -->
            <td class="columnDataNumber">{total['knowing']['multiple_choice']['total_questions']}</td>
            <td class="columnDataNumber">{total['knowing']['free_form']['total_questions']}</td>
            <td class="columnDataNumber">{total['understanding']['multiple_choice']['total_questions']}</td>
            <td class="columnDataNumber">{total['understanding']['free_form']['total_questions']}</td>
            <td class="columnDataNumber">{total['application']['multiple_choice']['total_questions']}</td>
            <td class="columnDataNumber">{total['application']['free_form']['total_questions']}</td>
            <td class="columnDataNumber">{total['adv_application']['multiple_choice']['total_questions']}</td>
            <td class="columnDataNumber">{total['adv_application']['free_form']['total_questions']}</td>
            <td class="columnDataNumber">{sum([total[kl]['multiple_choice']['total_questions'] for kl in ['knowing', 'understanding', 'application', 'adv_application']])}</td>
            <td class="columnDataNumber">{sum([total[kl]['free_form']['total_questions'] for kl in ['knowing', 'understanding', 'application', 'adv_application']])}</td>
            <td class="columnDataNumber">{total_score}</td>
            </tr>
            <tr>
            <td colspan="3">Tỷ lệ %</td>
            <!-- <td></td>
            <td></td> -->
            <td colspan="2">{knowing_score/total_score*100:.2f}%</td>
            <!-- <td></td> -->
            <td colspan="2">{understanding_score/total_score*100:.2f}%</td>
            <!-- <td></td> -->
            <td colspan="2">{application_score/total_score*100:.2f}%</td>
            <!-- <td></td> -->
            <td colspan="2">{adv_application_score/total_score*100:.2f}%</td>
            <!-- <td></td> -->
            <td colspan="2">{(knowing_score + understanding_score + application_score + adv_application_score)/total_score*100:.2f}%</td>
            <!-- <td></td> -->
            <td class="columnDataNumber"></td>
            </tr>
            <tr>
            <td colspan="3">Tỷ lệ chung %</td>
            <!-- <td></td>
            <td></td> -->
            <td colspan="4">{(knowing_score + understanding_score)/total_score*100:.2f}%</td>
            <!-- <td></td> -->
            <!-- <td></td> -->
            <!-- <td></td> -->
            <td colspan="4">{(application_score + adv_application_score)/total_score*100:.2f}%</td>
            <!-- <td></td> -->
            <!-- <td></td> -->
            <!-- <td></td> -->
            <td colspan="2">{(knowing_score + understanding_score + application_score + adv_application_score)/total_score*100:.2f}%</td>
            <!-- <td></td> -->
            <td class="columnDataNumber"></td>
            </tr>
        </tfoot>
    </body>
    </html>
    """
    return html_content

In [6]:
matrix_json_file = './matrix.json'
matrix_html_file = './matrix.html'


In [7]:
import json
with open(matrix_json_file, 'r', encoding='utf-8') as file:
    data = json.load(file)
    html = generating_exam_matrix(data)
    with open(matrix_html_file, "w") as matrix:
        matrix.write(html)

{'Bất đẳng thức. Bất phương trình bậc nhất một ẩn': {'BPT Bậc Nhất Một Ẩn': {'adv_application': {'free_form': {'num_questions': 0,
                                                                                                               'total_score': 0},
                                                                                                 'multiple_choice': {'num_questions': 0,
                                                                                                                     'total_score': 0}},
                                                                             'application': {'free_form': {'num_questions': 1,
                                                                                                           'total_score': 5},
                                                                                             'multiple_choice': {'num_questions': 0,
                                                                               

In [8]:
mt = json.load(open('matrix.json'))

In [9]:
pprint(mt)

[{'Chương hoặc Chủ đề': 'Bất đẳng thức. Bất phương trình bậc nhất một ẩn',
  'Loại câu hỏi': 'Trắc nghiệm',
  'Mức độ Nhận thức': 'Nhận biết',
  'Nội dung hoặc Đơn vị Kiến thức': 'Bất Đẳng Thức',
  'Số câu hỏi': 1,
  'Tổng số điểm': 4},
 {'Chương hoặc Chủ đề': 'Phương trình và hệ phương trình',
  'Loại câu hỏi': 'Trắc nghiệm',
  'Mức độ Nhận thức': 'Nhận biết',
  'Nội dung hoặc Đơn vị Kiến thức': 'Phương Trình Bậc Nhất',
  'Số câu hỏi': 1,
  'Tổng số điểm': 4},
 {'Chương hoặc Chủ đề': 'Căn thức',
  'Loại câu hỏi': 'Trắc nghiệm',
  'Mức độ Nhận thức': 'Nhận biết',
  'Nội dung hoặc Đơn vị Kiến thức': 'Căn Bậc Hai',
  'Số câu hỏi': 1,
  'Tổng số điểm': 4},
 {'Chương hoặc Chủ đề': 'Căn thức',
  'Loại câu hỏi': 'Trắc nghiệm',
  'Mức độ Nhận thức': 'Nhận biết',
  'Nội dung hoặc Đơn vị Kiến thức': 'Rút Gọn Biểu Thức Căn Bậc Hai',
  'Số câu hỏi': 1,
  'Tổng số điểm': 4},
 {'Chương hoặc Chủ đề': 'Căn thức',
  'Loại câu hỏi': 'Trắc nghiệm',
  'Mức độ Nhận thức': 'Nhận biết',
  'Nội dung hoặc Đơn

In [10]:
len(mt)

21

In [11]:
l = list((i["Chương hoặc Chủ đề"], i["Nội dung hoặc Đơn vị Kiến thức"], i["Mức độ Nhận thức"], i["Loại câu hỏi"]) for i in mt)
len(l)

21

In [12]:
len(set(l))

15